In [ ]:
# ref: https://data-flair.training/blogs/python-mini-project-speech-emotion-recognition/

In [1]:
pip install librosa soundfile numpy sklearn pyaudio


  Using cached PyAudio-0.2.11.tar.gz (37 kB)

  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio: started
    Running setup.py install for pyaudio: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\STRATUS-Research\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\STRATUS-Research\\AppData\\Local\\Temp\\pip-install-e9o8g57k\\pyaudio\\setup.py'"'"'; __file__='"'"'C:\\Users\\STRATUS-Research\\AppData\\Local\\Temp\\pip-install-e9o8g57k\\pyaudio\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\STRATUS-Research\AppData\Local\Temp\pip-wheel-eqwdqlst'
       cwd: C:\Users\STRATUS-Research\AppData\Local\Temp\pip-install-e9o8g57k\pyaudio\
  Complete output (9 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  copying src\pyaudio.py -> build\lib.win-amd64-3.8
  running build_ext
  building '_portaudio' extension
  error: Microsoft Visual C++ 14.0 or greater is r

In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

2. Define a function extract_feature to extract the mfcc, chroma, and mel features from a sound file. This function takes 4 parameters- the file name and three Boolean parameters for the three features:

mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
chroma: Pertains to the 12 different pitch classes
mel: Mel Spectrogram Frequency


### STEP 1
Open the sound file with soundfile.SoundFile using with-as so it’s automatically closed once we’re done. Read from it and call it X. Also, get the sample rate. If chroma is True, get the Short-Time Fourier Transform of X.

Let result be an empty numpy array. Now, for each feature of the three, if it exists, make a call to the corresponding function from librosa.feature (eg- librosa.feature.mfcc for mfcc), and get the mean value. Call the function hstack() from numpy with result and the feature value, and store this in result. hstack() stacks arrays in sequence horizontally (in a columnar fashion). Then, return the result.

In [5]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

3. Now, let’s define a dictionary to hold numbers and the emotions available in the RAVDESS dataset, and a list to hold those we want- calm, happy, fearful, disgust.

In [7]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']
emotions

{'01': 'neutral',
 '02': 'calm',
 '03': 'happy',
 '04': 'sad',
 '05': 'angry',
 '06': 'fearful',
 '07': 'disgust',
 '08': 'surprised'}

4. Now, let’s load the data with a function load_data() – this takes in the relative size of the test set as parameter. x and y are empty lists; we’ll use the glob() function from the glob module to get all the pathnames for the sound files in our dataset. The pattern we use for this is: “D:\\DataFlair\\ravdess data\\Actor_*\\*.wav”. This is because our dataset looks like this:

So, for each such path, get the basename of the file, the emotion by splitting the name around ‘-’ and extracting the third value:
Using our emotions dictionary, this number is turned into an emotion, and our function checks whether this emotion is in our list of observed_emotions; if not, it continues to the next file. It makes a call to extract_feature and stores what is returned in ‘feature’. Then, it appends the feature to x and the emotion to y. So, the list x holds the features and y holds the emotions. We call the function train_test_split with these, the test size, and a random state value, and return that.

In [49]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [51]:
x_train,x_test,y_train,y_test= load_data(.25)

6. Observe the shape of the training and testing datasets:

In [42]:
print("train\n", x_train, "\ntest\n", x_test)
print(x_train)

train
 [[-5.21694946e+02  5.43121452e+01 -1.12595196e+01 ...  2.16407017e-04
   1.40978998e-04  1.14794449e-04]
 [-4.52471008e+02  1.81078854e+01 -2.65877132e+01 ...  5.50175435e-04
   5.70777862e-04  4.87014215e-04]
 [-5.58079468e+02  5.66460762e+01  4.79677868e+00 ...  2.85093585e-04
   1.95071072e-04  1.24067476e-04]
 ...
 [-6.00510132e+02  5.31514320e+01  8.95492458e+00 ...  2.66101619e-04
   1.20797216e-04  9.05639026e-05]
 [-5.49937622e+02  2.05582409e+01 -1.82497749e+01 ...  2.15354215e-04
   1.42610632e-04  5.24237585e-05]
 [-6.91638672e+02  7.27555847e+01  5.44958353e+00 ...  1.47388891e-05
   6.60104979e-06  2.86574937e-06]] 
test
 [[-5.57380188e+02  5.61003227e+01 -8.54372311e+00 ...  5.85538401e-05
   3.56250130e-05  2.51964248e-05]
 [-4.99206329e+02  4.36633873e+00 -3.76205444e+01 ...  1.71078209e-04
   1.43228433e-04  9.51329712e-05]
 [-5.68905151e+02  3.63499107e+01 -5.40141392e+00 ...  6.29053276e-04
   2.47150223e-04  1.85339581e-04]
 ...
 [-6.47894592e+02  4.48749809e

In [43]:
 # 7. And get the number of features extracted.
print(f'Features: {x_train.shape[1]}')


Features: 180


8. Now, let’s initialize an MLPClassifier. This is a Multi-layer Perceptron Classifier; it optimizes the log-loss function using LBFGS or stochastic gradient descent. Unlike SVM or Naive Bayes, the MLPClassifier has an internal neural network for the purpose of classification. This is a feedforward ANN model.

In [44]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01,
                    batch_size=256, 
                    epsilon=1e-08,
                    hidden_layer_sizes=(300,),
                    learning_rate='adaptive', 
                    max_iter=500)


In [45]:
# fit model
model.fit(x_train, y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [46]:
# #DataFlair - Predict for the test set
y_pred=model.predict(x_test)


11. To calculate the accuracy of our model, we’ll call up the accuracy_score() function we imported from sklearn. Finally, we’ll round the accuracy to 2 decimal places and print it out.

In [47]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 48.06%


In [ ]:
emotions[file_name.split("-")[2]]